In [1]:
import numpy as np
from scipy.spatial.distance import pdist, cdist

In [2]:
def parse_input(filename: str):
    beacons = []
    sensors = []
    man_dist = []
    with open(filename) as f:
        for row in f:
            row = row.rstrip().split(" ")
            sensor_x = int(row[2].split("=")[1][:-1])
            sensor_y = int(row[3].split("=")[1][:-1])
            beacon_x = int(row[8].split("=")[1][:-1])
            beacon_y = int(row[9].split("=")[1])
            beacons.append([beacon_x, beacon_y])
            sensors.append([sensor_x, sensor_y])
            man_dist.append(pdist([[sensor_x, sensor_y], [beacon_x, beacon_y]], metric="cityblock"))
    return np.stack(beacons).astype(int), np.stack(sensors).astype(int), np.stack(man_dist).astype(int).flatten()

In [3]:
def _get_char_repr(sensors, beacons, max_dist, x, y):
    dist_mat = cdist([[x, y]], sensors, metric="cityblock").flatten()
    if (x, y) in set(map(tuple, beacons)):
        return "B"
    elif (x, y) in set(map(tuple, sensors)):
        return "S"
    elif (dist_mat <= max_dist).any():
        return "#"
    else:
        return "."


def task_one_print(filename, y_row, verbose: bool = False):
    beacons, sensors, man_dist = parse_input(filename)
    min_x, min_y = sensors.min(axis=0)
    max_x, max_y = sensors.max(axis=0)
    max_man_dist = man_dist.max()
    sum_of_blocked = 0
    ret_str = ""
    if verbose:
        for _y in range(min_y - max_man_dist, y_row):
            ret_str += f"{_y:3d} "
            for _x in range(min_x - max_man_dist, max_x + max_man_dist + 1):
                ret_str += _get_char_repr(sensors, beacons, man_dist, _x, _y)
            ret_str += "\n"

    ret_str += f"{y_row:3d} "
    for _x in range(min_x - max_man_dist, max_x + max_man_dist + 1):
        inc = _get_char_repr(sensors, beacons, man_dist, _x, y_row)
        if inc == "#":
            sum_of_blocked += 1
        ret_str += inc
    ret_str += "\n"
    if verbose:
        for _y in range(y_row + 1, max_y + max_man_dist + 1):
            ret_str += f"{_y:3d} "
            for _x in range(min_x - max_man_dist, max_x + max_man_dist + 1):
                ret_str += _get_char_repr(sensors, beacons, man_dist, _x, _y)
            ret_str += "\n"

        prev_row_100 = "    "
        prev_row_10 = "    "
        prev_row_1 = "    "
        for _x in range(min_x - max_man_dist, max_x + max_man_dist + 1):
            if _x % 5 == 0:
                num_str = f"{_x:3d}"
                prev_row_100 += num_str[0]
                prev_row_10 += num_str[1]
                prev_row_1 += num_str[2]
            else:
                prev_row_100 += " "
                prev_row_10 += " "
                prev_row_1 += " "

        print(min_x-max_man_dist, max_x+max_man_dist)
        print(prev_row_100)
        print(prev_row_10)
        print(prev_row_1)
        print(ret_str)
    return sum_of_blocked

In [4]:
task_one_print("test-input.txt", 10, verbose=True)

-10 30
    -                                        
    1    -              1    1    2    2    3
    0    5    0    5    0    5    0    5    0
-10 ............#............................
 -9 ...........###...........................
 -8 ..........#####..........................
 -7 .........#######.........................
 -6 ........#########.............#..........
 -5 .......###########...........###.........
 -4 ......#############.........#####........
 -3 .....###############.......#######.......
 -2 ....#################.....#########......
 -1 ...###################.#.###########.....
  0 ..##########S########################....
  1 ...###########################S#######...
  2 ....###################S#############....
  3 .....###################SB##########.....
  4 ......#############################......
  5 .......###########################.......
  6 ........#########################........
  7 .........#########S#######S#####.........
  8 ..........#############

26

In [5]:
def task_one(filename: str, y_row: int, verbose: bool = False):
    beacons, sensors, man_dist = parse_input(filename)
    min_x, min_y = sensors.min(axis=0)
    max_x, max_y = sensors.max(axis=0)
    max_man_dist = man_dist.max()
    dist_mat = cdist(
        [
            [_x, y_row] for _x in range(min_x - max_man_dist, max_x + max_man_dist + 1)
        ],
        sensors,
        metric="cityblock"
    )
    mask = (dist_mat <= man_dist).any(axis=1)
    
    mask2 = np.stack(list(set(map(tuple, beacons))))[:, 1] == y_row
    mask3 = np.stack(list(set(map(tuple, sensors))))[:, 1] == y_row
    mask_sum = mask.sum()
    mask2_sum = mask2.sum()

    if verbose:
        ret_str = ""
        for _x, m in zip(range(min_x - max_man_dist, max_x + max_man_dist + 1), mask):
            if (_x, y_row) in set(map(tuple, beacons)):
                ret_str += "B"
            elif (_x, y_row) in set(map(tuple, sensors)):
                ret_str += "S"
            elif m:
                ret_str += "#"
            else:
                ret_str += "."
        print(ret_str)
    return mask_sum - mask2_sum - mask3.sum()

In [6]:
print(task_one("test-input.txt", 10, verbose=True))
print(task_one("input.txt", 2000000))

........####B######################......
26
4793062


In [7]:
# rather inefficient but gets the job done
from tqdm import tqdm
from itertools import product
def task_two(filename: str, bound: tuple[int, int] = (0, 4000000)):
    beacons, sensors, man_dist = parse_input(filename)
    for sensor, d in tqdm(zip(sensors, man_dist)):
        for dx in range(d+2):
            dy = d + 1 - dx
            for step_x, step_y in product([-1, 1], [-1, 1]):
                pos = sensor + np.array([dx*step_x, dy*step_y])
                if not (bound[0] <= pos[0] <= bound[1] and bound[0] <= pos[1] <= bound[1]):
                    continue
                elif _get_char_repr(sensors, beacons, man_dist, pos[0], pos[1]) == ".":
                    return pos, pos[0] * 4000000 + pos[1]

print(task_two("test-input.txt", bound=[0, 20]))
print(task_two("input.txt"))

3it [00:00, 651.59it/s]
0it [00:00, ?it/s]

(array([14, 11]), 56000011)


1it [02:39, 159.56s/it]

(array([2706598, 3253551]), 10826395253551)
